# Exploratory Data Analysis (EDA) for Food Reviews Dataset

## Overview of the Datasets


### Load the datasets

In [ ]:
# Import required libraries
import pandas as pd

# Load the data from the CSV and TXT files
bad_lines = []
def collect_bad_line(line):
    bad_lines.append(line)
    return None  # skip

nutrition_df = pd.read_csv('data/Nutrition.csv', sep=";")
restaurants_df = pd.read_csv('data/Restaurants.csv', sep=";")
reviews_df = pd.read_csv('data/Reviews.txt', sep="\t")
recipes_df = pd.read_csv("data/Recipes.csv", sep=";", engine="python", on_bad_lines=collect_bad_line)

print("loaded:", recipes_df.shape)
print("bad lines captured:", len(bad_lines))
print("example bad line:", bad_lines[0][:2000] if bad_lines else "none")


### Recipes Dataset

In [ ]:
print('====== Recipes Dataset ======')
print('Number of rows: ', recipes_df.shape[0])
print('Number of columns: ', recipes_df.shape[1])
print('Number of Unique Recipe Categories: ', len(list(set(recipes_df['RecipeCategory'].unique()))))
print('Number of Unique DatePublished: ', len(list(set(recipes_df['DatePublished'].unique()))))
print('Number of duplicates: ', recipes_df.duplicated().sum())
print('Number of Null Values: ', recipes_df.isnull().sum().sum())
print('Number of rows with null values: ', len(recipes_df[recipes_df.isnull().any(axis=1)].index))
print('Columns with Null Values: ', recipes_df.isnull().sum())
print('\n')
recipes_df.head()


### Nutrition Dataset


In [ ]:
print('====== Nutrition Dataset ======')
print('Number of rows: ', nutrition_df.shape[0])
print('Number of columns: ', nutrition_df.shape[1])
unique_recipe_categories = list(set(nutrition_df['RecipeCategory'].unique()))
print('Number of Unique Recipe Categories: ', len(unique_recipe_categories))
print(unique_recipe_categories)
if 'Fast Food' in unique_recipe_categories: print('Hawaiian is in the list')
print('Number of duplicates: ', nutrition_df.duplicated().sum())
print('Number of Null Values: ', nutrition_df.isnull().sum().sum())
print('Number rows with null values: ', [r for r in nutrition_df[nutrition_df.isnull().any(axis=1)].index])
print('Columns with Null Values: ', nutrition_df.isnull().sum())
print('\n')
nutrition_df.head()

### Restaurants Dataset

In [ ]:
print('====== Restaurants Dataset ======')
print('Number of rows: ', restaurants_df.shape[0])
print('Number of columns: ', restaurants_df.shape[1])
print('Number of Unique Countries: ', len(list(set(restaurants_df['Country'].unique()))))
print('Number of Unique Cuisines: ', len(list(set(restaurants_df['Cuisines'].unique()))))
print('Number of duplicates: ', restaurants_df.duplicated().sum())
print('Number of Null Values: ', restaurants_df.isnull().sum().sum())
print('The rows of the null values: ', [r for r in restaurants_df[restaurants_df.isnull().any(axis=1)].index])
print('\n')
restaurants_df.head()

In [ ]:
for index, row in restaurants_df.iterrows():
    q = row['Cuisines']
    for u in unique_recipe_categories:
        if u == q:
            print(f"Found a match in row {index} with category {u} and cuisine {q}")

### Reviews Dataset
This is the unstructured dataset that contains the reviews from users. the reviews are supposed to be on a single line, however this is not the case and a single review may be split up into more than one line.

In [ ]:
print('====== Reviews Dataset ======')
print('Number of rows: ', reviews_df.shape[0])
print('Number of columns: ', reviews_df.shape[1])
print('Number of Unique Authors: ', len(list(set(reviews_df['AuthorId'].unique()))))
print('Number of duplicates: ', reviews_df.duplicated().sum())
print('Number of Null Values: ', reviews_df.isnull().sum().sum())
print('Number of rows with null values: ', len(reviews_df[reviews_df.isnull().any(axis=1)].index))
print('Columns with Null Values: \n', reviews_df.isnull().sum())
print('\n')
reviews_df.head(20)

In [ ]:
# reviews that have recipe ids that are not present in the recipes dataset
invalid_reviews = reviews_df[reviews_df['RecipeId'].isin(recipes_df['RecipeId'])]
print('Number of reviews with invalid recipe ids: ', invalid_reviews.shape[0])

## Cleaning the Datasets
With every single dataset we will have to clean it in order to make it suitable for the construction of the knowledge graph. In all datasets duplicated rows are removed. Null values are treated accordingly based on the importance of the attribute. Also, only recipes which have nutritional information are kept, therefore, keeping only recipes that are present in both the Recipes and Nutrition datasets.

#### Relevant Columns in each dataset:
- For the **Recipe dataset**, <u>remove</u> the `RecipeYield`, `RecipeServings`, and `Images` columns as they are not relevant to the task. Replace the null values in the `CookTime` column with the median cook time. Also, <u>remove</u> the rows that have the null values in the `RecipeCategory` and `Keywords` column as they are crucial attributes for the knowledge graph.

- In the **Nutrition dataset**, <u>keep</u> all the columns as they are relevant to the construction of the knowledge graph. But <u>remove</u> the rows that do not have a `RecipeCategory` value and nutritional values.

- In the **Restaurants dataset**, <u>remove</u> the `Is delivering now`, `Switch to order menu`, `Price range`, `Rating color`, `Rating text`, `Longitude`, `Latitude` and `Nummber of dishes in cuisines` columns as they are not relevant to the task.

- The **Reviews dataset**, which is unstructured, is preprocessed to deal with all the reviews that are split into multiple lines and columns. All columns are <u>kept</u>.

In [ ]:
# Remove duplicates from all the datasets
recipes_df.drop_duplicates(inplace=True)
restaurants_df.drop_duplicates(inplace=True)
nutrition_df.drop_duplicates(inplace=True)

# Remove irrelevant columns from the datasets
recipes_df.drop(columns=["RecipeYield", "RecipeServings", "Images"], inplace=True)
restaurants_df.drop(columns=["Is delivering now", "Switch to order menu", "Price range", "Nummber of dishes in cuisines", "Rating color", "Rating text", "Longitude", "Latitude"], inplace=True)

# Remove rows with null values in the RecipeCategory and Keywords columns in the Recipes dataset
recipes_df.dropna(subset=['RecipeCategory', 'Keywords', 'RecipeIngredientQuantities'], inplace=True)
nutrition_df.dropna(inplace=True) # remove rows with null values in the Nutrition dataset

# Remove all the recipes in the Recipes dataset that are not present in the Nutrition dataset
recipes_df = recipes_df[recipes_df['Name'].isin(nutrition_df['Name'])]

In [ ]:
# Clean the cook and prep times of the Recipes dataset
def clean_cook_time_column(df, column):
    for index, row in df.iterrows(): # iterate through each row in the dataframe
        minutes = 0 # initialise the minutes variable
        time_str = row[column] # get the cook time string
        if pd.isna(time_str): # if the cook time value is null, skip the row
            df.at[index, column] = str(0)
            continue
        time_str = time_str.replace('PT', '') # remove the 'PT' prefix from the cook time values
        for char in time_str: # iterate through each character in the cook time string
            if char == "H":
                minutes += int(time_str[:time_str.index(char)]) * 60 # add the hours to the minutes variable
                time_str = time_str[time_str.index(char)+1:]
            elif char == "M":
                minutes += int(time_str[:time_str.index(char)]) # add the minutes to the minutes variable
        df.at[index, column] = str(minutes) # replace the cook time string with the minutes variable
    df[column] = pd.to_numeric(df[column], downcast='float') # convert the cook time values to numeric
    df[column].fillna(df[column].median()) # replace null cook time values with median time

# Convert CookTime and PrepTime to numeric and replace null values with median
clean_cook_time_column(recipes_df, 'CookTime')
clean_cook_time_column(recipes_df, 'PrepTime')

#### Clean the Reviews dataset

In [ ]:
# String Type Constants
NL = 0
DT = 1
Z = 2
NULL = 3


def is_datetime_string(s):
    if isinstance(s, str) and len(s) == 20 and s[4] == '-' and s[7] == '-':
        return True
    return False

def clean_line_and_split(line):
    # remove the double quotes from the line and split it by tab character
    line = line.replace('"""', "").replace('""', "").replace('"', "")
    split_line = line.split("\t") # split the line based on the tab character
    split_line = [s.strip() for s in split_line] # remove leading and trailing whitespace from each element
    return split_line

def get_string_types_from_split_line(row: list[str]):
    string_types = []
    for i in row:
        if i == '' or i is None or i.lower() == 'nan':
            string_types.append(NULL)
        elif is_datetime_string(i):
            string_types.append(DT)
        elif i.isdigit():
            string_types.append(Z)
        else:
            string_types.append(NL)
    return string_types

# Check if a row is in the correct order of the column types
def check_correct_row_order(row: list[str]):
    st = get_string_types_from_split_line(row)
    if len(st) == len(row) == 7 and st[0] == Z and st[1] == Z and st[2] == Z and st[3] == NL and st[4] == NL and st[5] == DT and st[6] == DT:
        return True
    return False


# Function to make the row from an initial line. It iteratively builds and cleans rows until the correct order is achieved
def make_row(all_lines, line_idx, line_string):
    sl = clean_line_and_split(line_string)
    # print(sl)
    # Merge the reviews that have been split into multiple lines or columns
    row = [None for _ in range(7)] # new row to store the cleaned data
    review = "" # string to store the review text
    date1, date2 = None, None # strings to store the first and second datetime
    for n, col in enumerate(sl):
        if col.isdigit() and n < 3: # the first three columns ReviewID, RecipeID, AuthorID should be numeric values
                row[n] = col
        elif n == 3 and not col.isdigit() and not is_datetime_string(col): # the fourth should be the AuthorName column
            row[n] = col
        else:
            if not is_datetime_string(col):
                review += " " + col
            else:
                if not date1 and col != '':
                    date1 = col
                elif not date2 and date1 and col != '':
                    date2 = col
    st_1 = get_string_types_from_split_line(sl)

    # If the previous row had only natural language, empty values and integers and no datetime strings, then we can merge the next
    # row with the current row depending on whether the next row has only datetime strings or natural language and empty values
    next_row_idx = line_idx + 1
    while date1 is None and date2 is None and next_row_idx < len(all_lines):
        if st_1.count(NL) + st_1.count(NULL) + st_1.count(Z) == len(st_1):
        # if not check_correct_row_order(sl):
        #     print('next row needs to be checked')
            # Get the next line
            next_line = clean_line_and_split(all_lines[next_row_idx])
            st_2 = get_string_types_from_split_line(next_line)
            # If the next row ha sonly natural language, then merge the text into the review column
            if st_2.count(NL) + st_2.count(NULL) == len(st_2):
                review += " ".join(next_line)
            # If the next row has only datetime strings, then merge the datetime into the date columns
            if st_2.count(DT) + st_2.count(NULL) == len(st_2):
                extracted_dates = [i for i in next_line if is_datetime_string(i)]
                date1 = extracted_dates[0]
                date2 = extracted_dates[1] if len(extracted_dates) > 1 else date1
            # If the next row has both natural language and datetime strings
            if st_2.count(NL) + st_2.count(DT) + st_2.count(NULL) == len(st_2) and st_2.count(DT) != 0 and st_2.count(NL) != 0:
                extracted_dates = [i for i in next_line if is_datetime_string(i)]
                date1 = extracted_dates[0]
                date2 = extracted_dates[1] if len(extracted_dates) > 1 else date1
                review += " ".join([i for i in next_line if not is_datetime_string(i) and i != '' and i is not None])
        next_row_idx += 1

    row[4] = review.strip()
    row[5] = date1
    row[6] = date2

    return row


clean_reviews = []
with open('data/Reviews.txt', 'r') as f:
    lines = f.readlines()
    # print('First 5 lines of the Reviews.txt file:')
    for idx, line in enumerate(lines):
        if idx == 0: continue # skip the header line
        # print(f'======== Line {idx} =======')
        row = make_row(lines, idx, line)
        if check_correct_row_order(row):
            clean_reviews.append(row)

        # print(st_1)
        # print(row)


clean_reviews = pd.DataFrame(clean_reviews, columns=['ReviewId', 'RecipeId', 'AuthorId', 'AuthorName', 'Review', 'DateSubmitted', 'DateModified'])
print(f'Reduced to {len(clean_reviews)} rows from {reviews_df.shape[0]}')
print('Number of rows: ', clean_reviews.shape[0])
print('Number of columns: ', clean_reviews.shape[1])
print('Number of Unique Authors: ', len(list(set(clean_reviews['AuthorId'].unique()))))
print('Number of duplicates: ', clean_reviews.duplicated().sum())
print('Number of Null Values: ', clean_reviews.isnull().sum().sum())
print('Number of rows with null values: ', len(clean_reviews[clean_reviews.isnull().any(axis=1)].index))
print('Columns with Null Values: \n', clean_reviews.isnull().sum())
print('\n')
clean_reviews.head()


## Save datasets

In [11]:
recipes_df.to_csv('data/cleaned_recipes.csv', index=False)
nutrition_df.to_csv('data/cleaned_nutrition.csv', index=False)
restaurants_df.to_csv('data/cleaned_restaurants.csv', index=False)
clean_reviews.to_csv('data/cleaned_reviews.csv', index=False)